# Import packages and define functions

In [ ]:
from ntuples_to_dfs import *
from template_fit import *
from default_values import *

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
def checkAntiisoSlices(isovar, fitvar, isohist, mchist, mcerr, antiisoslices, datadf, binEdges, purityRange):
    chi2dofs = []
    purities = []
    
    for antiisoslice in antiisoslices:
        antiisodf = applyCut(datadf, '{0}>{1} and {0}<{2}'.format(isovar, *antiisoslice))
        antiisohist = np.histogram(antiisodf[fitvar], bins=binEdges, weights=antiisodf['weights'])[0]

        tf = TemplateFit(isohist, None, mchist, mcerr, antiisohist, None, binEdges, verbosity=0)
        chi2dof = tf.chi2 / tf.dof
        purity = tf.getPurity(*purityRange)[0]

        chi2dofs.append(chi2dof)
        purities.append(purity)

    return chi2dofs, purities


def getBand(antiisoslices, values, antiisoregion):
    minValue = np.inf
    maxValue = -np.inf
    
    for (antiisoslice, value) in zip(antiisoslices, values):
        if antiisoslice[0] >= antiisoregion[0] and antiisoslice[1] <= antiisoregion[1]:
            minValue = min(minValue, value)
            maxValue = max(maxValue, value)
            
    return minValue, maxValue


def addInfo(ptrange, ylabel, showLegend=True):
    ax = plt.gca()
    xlim = ax.get_xlim()
    ax.axvspan(xlim[0], 1.5, color='k', alpha=0.4)
    ax.set_xlim(xlim)
    ax.axvspan(antiisocut[0], antiisocut[1], color='g', alpha=0.1)
    plt.xlabel('Anti-isolation cut [GeV]')
    plt.ylabel(ylabel)

    if showLegend:
        plt.plot([], [], ' ', label='{0} < pT < {1} GeV'.format(*ptrange))
        plt.legend(loc=4)
        

def plotAntiisoSlices(fulldatadf, fullmcdf, ptrange):
    antiisoslices = [(x, x + 2.0) for x in np.arange(-3, 25, 0.5)]
    antiisocenters = [np.mean(s) for s in antiisoslices]
    antiisowidths = [(s[1] - s[0]) / 2.0 for s in antiisoslices]

    ptcut = '{0}>{1} and {0}<{2}'.format('cluster_pt', *ptrange)
    datadf = applyCut(fulldatadf, ptcut)
    mcdf = applyCut(fullmcdf, ptcut)

    isodf = applyCut(datadf, isocuttext)
    mcisodf = applyCut(mcdf, isocuttext)

    plt.figure(figsize=(18, 6))

    for fitvar, color in zip([nn1, lambda2], ['b', 'r']):
        binEdges, purityRange, _ = varinfo[fitvar]

        isohist = np.histogram(isodf[fitvar], bins=binEdges, weights=isodf['weights'])[0]
        mchist, mcerr = getHistAndErr(mcisodf, fitvar, binEdges)

        chi2dofs, purities = checkAntiisoSlices(isovar, fitvar, isohist, mchist, mcerr, antiisoslices, datadf, binEdges, purityRange)
        chi2band = getBand(antiisoslices, chi2dofs, antiisocut)
        purband = getBand(antiisoslices, purities, antiisocut)

        print '{0} for {1} < pT < {2}: {3:1.2f} to {4:1.2f}; {5:2.2f}\%'.format(fitvar, ptrange[0], ptrange[1], purband[0], purband[1], 100*(purband[1]-purband[0])/np.sqrt(12))

        label = '{1} < {0} < {2}'.format(fitvar, *purityRange)

        plt.subplot(121)
        plt.errorbar(antiisocenters, chi2dofs, xerr=antiisowidths, fmt='none', ecolor=color, label=label)
        addInfo(ptrange, 'Chi2/dof', showLegend=False)

        plt.subplot(122)
        plt.errorbar(antiisocenters, purities, xerr=antiisowidths, fmt='none', ecolor=color, label=label)
        ax = plt.gca()
        ax.axhspan(purband[0], purband[1], color=color, alpha=0.2)
        addInfo(ptrange, 'Purity', showLegend=True)    


# p-Pb
## Process ntuples

In [ ]:
ppbdatadf = applyCuts(getData(pPbdataFiles, ntupledir), fullpt)
ppbgjmcdf = applyCuts(getData(pPbgjmcFiles, ntupledir), fullpt)

## Make plots

In [ ]:
plotAntiisoSlices(ppbdatadf, ppbgjmcdf, gjpt)
plotAntiisoSlices(ppbdatadf, ppbgjmcdf, ghpt)

# pp
## Process ntuples

In [ ]:
ppdatadf = applyCuts(getData(ppdataFiles, ntupledir), fullpt)
ppgjmcdf = applyCuts(getData(ppgjmcFiles, ntupledir), fullpt)

## Make plots

In [ ]:
plotAntiisoSlices(ppdatadf, ppgjmcdf, gjpt)
plotAntiisoSlices(ppdatadf, ppgjmcdf, ghpt)